In [ ]:
!unzip -q archive.zip -d dataset

print("Unzipping complete. Here is the folder structure:")
!ls dataset

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# Settings
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

print("Libraries imported.")

In [ ]:
def load_data(root_path):
    images = []
    masks = []

    print("Scanning for images and masks...")

    for root, dirs, files in os.walk(root_path):
        for file in files:
            # We look for original images (not masks) first
            if 'mask' not in file and file.lower().endswith('.png'):

                # Construct the path to the image
                img_path = os.path.join(root, file)

                # Construct the expected path to the mask
                # (Assumes mask has same name but adds '_mask')
                mask_name = file.replace('.png', '_mask.png')
                mask_path = os.path.join(root, mask_name)

                # Only proceed if the mask actually exists
                if os.path.exists(mask_path):
                    # Load and Resize Image
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

                    # Load and Resize Mask (Load as grayscale)
                    mask = cv2.imread(mask_path, 0)
                    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
                    mask = np.expand_dims(mask, axis=-1) # Add depth for AI processing

                    images.append(img)
                    masks.append(mask)

    return np.array(images), np.array(masks)

# Load the data from the folder we created in Cell 1
X, y = load_data('./dataset')

print(f"Successfully loaded {len(X)} image/mask pairs.")

# Normalize the pixel data (0 to 1 range)
X = X / 255.0
y = y / 255.0 # Masks become 0 or 1

# Split into Training (90%) and Test (10%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print("Data ready for training.")

In [ ]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    # --- ENCODER (Contracting Path) ---
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    # --- BOTTLENECK ---
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    # --- DECODER (Expansive Path) ---
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same')(c9)

    # Output Layer: Sigmoid gives a probability (0-1) for every pixel
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_unet((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
print("Model built successfully.")

In [ ]:
print("Starting training...")
history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    batch_size=16,
                    epochs=15,
                    verbose=1)
print("Training finished.")

In [ ]:
# 1. Make predictions
preds = model.predict(X_test, verbose=0)
# Convert probabilities to strictly 0 or 1 (Tumor or No Tumor)
preds_thresholded = (preds > 0.5).astype(np.uint8)

# 2. Function to plot
def plot_results(num_samples=3):
    for i in range(num_samples):
        ix = np.random.randint(0, len(X_test)) # Pick random image

        plt.figure(figsize=(12, 4))

        # Plot Original Ultrasound
        plt.subplot(1, 3, 1)
        plt.imshow(X_test[ix])
        plt.title("Original Ultrasound")
        plt.axis('off')

        # Plot True Mask (Ground Truth)
        plt.subplot(1, 3, 2)
        plt.imshow(y_test[ix].squeeze(), cmap='gray')
        plt.title("Actual Tumor (Doctor)")
        plt.axis('off')

        # Plot AI Prediction
        plt.subplot(1, 3, 3)
        plt.imshow(preds_thresholded[ix].squeeze(), cmap='gray')
        plt.title("AI Prediction")
        plt.axis('off')

        plt.show()

# 3. Show the plots
plot_results()

This cell first deletes any old failed attempts to keep your workspace clean.

It unzips archive.zip into a folder called dataset_intense.

It checks the file size first to warn you if the upload failed again.

In [ ]:
import os
import shutil

# 1. Clean up previous runs (Optional but recommended)
if os.path.exists('dataset_intense'):
    shutil.rmtree('dataset_intense')

# 2. Check if archive.zip exists and has data
file_path = 'archive.zip'
if not os.path.exists(file_path):
    print("ERROR: 'archive.zip' not found. Please upload it to the Files tab on the left.")
elif os.path.getsize(file_path) < 1000:
    print("ERROR: 'archive.zip' is too small (looks empty). Please delete and re-upload.")
else:
    # 3. Unzip
    print("Unzipping archive.zip...")
    !unzip -q archive.zip -d dataset_intense
    print("Unzipping complete.")

    # 4. Verify what we have
    print("\nRoot folder contents:")
    !ls dataset_intense

We are adding BatchNormalization, Dropout, and Callbacks to our imports. These are the tools for the "intense" training.

In [ ]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# Configuration
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

print("Libraries imported. GPU Available: ", len(tf.config.list_physical_devices('GPU')) > 0)

Smart Data Loading
Notes for Colab:

This is the same robust loader as before. It scans the dataset_intense folder.

It normalizes images (dividing by 255) immediately to save RAM.

In [ ]:
def load_data(root_path):
    images = []
    masks = []

    print("Scanning for data...")
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if 'mask' not in file and file.lower().endswith('.png'):
                img_path = os.path.join(root, file)
                mask_name = file.replace('.png', '_mask.png')
                mask_path = os.path.join(root, mask_name)

                if os.path.exists(mask_path):
                    # Load Image
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
                    images.append(img)

                    # Load Mask (Grayscale)
                    mask = cv2.imread(mask_path, 0)
                    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
                    mask = np.expand_dims(mask, axis=-1)
                    masks.append(mask)

    return np.array(images), np.array(masks)

# Load data
X, y = load_data('./dataset_intense')

# Check if data loaded correctly
if len(X) == 0:
    print("ERROR: No images found. Check your zip file structure.")
else:
    print(f"Loaded {len(X)} images and masks.")

    # Normalize
    X = X / 255.0
    y = y / 255.0

    # Split Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    print("Data normalized and split ready for intense training.")

Filters: We start at 32 filters (instead of 16) for deeper feature detection.

Dropout (0.2): Discards 20% of neurons randomly to prevent memorization.

BatchNormalization: Added after every convolution block. This stabilizes the math inside the network, allowing it to learn much faster without getting "confused."

In [ ]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # --- Contraction Path (Encoder) ---
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    # --- Bottleneck ---
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # --- Expansive Path (Decoder) ---
    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    c9 = BatchNormalization()(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
print("Intense Model Built.")

EarlyStopping: If the model stops improving for 10 epochs, training stops automatically. This saves time and prevents overfitting.

ReduceLROnPlateau: If the model gets "stuck", this lowers the learning rate (makes the steps smaller) to help it find the optimal solution.

Epochs: Set to 50. Don't worry, EarlyStopping will cut it short if it finishes early.

In [ ]:
# Callbacks
early_stopper = EarlyStopping(patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

print("Starting intense training...")
history = model.fit(X_train, y_train,
                    batch_size=16,
                    epochs=50,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopper, reduce_lr])

print("Training finished.")

This visualizer creates a large, clear comparison.

It converts the AI's "probability map" into a crisp Black/White mask using a 0.5 threshold.

In [ ]:
# Predict
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

def plot_random_sample():
    ix = np.random.randint(0, len(X_test))

    fig, ax = plt.subplots(1, 3, figsize=(20, 8))

    # Original
    ax[0].imshow(X_test[ix])
    ax[0].set_title("Ultrasound", fontsize=16)
    ax[0].axis('off')

    # Ground Truth
    ax[1].imshow(y_test[ix].squeeze(), cmap='gray')
    ax[1].set_title("Doctor's Label (Ground Truth)", fontsize=16)
    ax[1].axis('off')

    # Prediction
    ax[2].imshow(preds_test_t[ix].squeeze(), cmap='gray')
    ax[2].set_title("AI Prediction", fontsize=16)
    ax[2].axis('off')

    plt.show()

# Run this line multiple times to see different images
plot_random_sample()
plot_random_sample()

In [ ]:
# Save the model architecture and weights
model.save('ultrasound_unet_model.h5')
print("Model saved as 'ultrasound_unet_model.h5'")

# Download it to your local computer (Optional but recommended)
from google.colab import files
files.download('ultrasound_unet_model.h5')

Calculate Medical Metrics (For your Report)
Standard "Accuracy" is misleading in cancer detection because most of the image is black background. If the AI predicts "all black," it might still get 90% accuracy but miss the tumor entirely.

You need Dice Coefficient and IoU (Intersection over Union). These measure the overlap between the Doctor's mask and the AI's mask.

In [ ]:
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def iou_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + 1) / (union + 1)

# Calculate on the test set
# Note: We use the raw predictions (probabilities), not the thresholded ones, for smoother calculation
results = model.evaluate(X_test, y_test, verbose=0)
print(f"Standard Accuracy: {results[1]*100:.2f}%")

# Calculate specific Dice/IoU
predictions = model.predict(X_test, verbose=0)
dice = dice_coef(tf.cast(y_test, tf.float32), tf.cast(predictions, tf.float32))
iou = iou_coef(tf.cast(y_test, tf.float32), tf.cast(predictions, tf.float32))

print(f"Dice Score (Overlap Accuracy): {dice.numpy():.4f}")
print(f"IoU Score (Intersection over Union): {iou.numpy():.4f}")

Test on a Single New Image (Simulation)
Imagine you are a doctor and you just took a new ultrasound. You want to see if the AI can handle a raw image that it has never seen before (and that doesn't have a mask yet).

In [ ]:
from google.colab import files
from PIL import Image

print("Upload a new ultrasound image (png/jpg):")
uploaded = files.upload()

for fn in uploaded.keys():
    # 1. Load the image
    img = Image.open(fn).convert('RGB') # Ensure it's 3 channels
    img = img.resize((128, 128))
    img_array = np.array(img) / 255.0  # Normalize
    img_input = np.expand_dims(img_array, axis=0) # Add batch dimension (1, 128, 128, 3)

    # 2. Predict
    prediction = model.predict(img_input)
    prediction_mask = (prediction > 0.5).astype(np.uint8)

    # 3. Plot
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(img_array)
    plt.title("Uploaded Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(prediction_mask[0, :, :, 0], cmap='gray')
    plt.title("AI Generated Mask")
    plt.axis('off')

    plt.show()

TypeError: 'NoneType' object is not subscriptable